In [78]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import matplotlib.patches as patches
import cv2
import numpy as np
from operator import itemgetter, attrgetter

In [79]:
class box : 
    def __init__(self, x1, y1, x2, y2) :
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.children = []
        
class column : 
    def __init__(self, min_num, max_num, id) :
        self.min_num = min_num
        self.max_num = max_num
        self.id = id
        self.lines = []
    def add_line(self, line) : 
        self.lines.append(line)
        if line.x1 < self.min_num : 
            self.min_num = line.x1
        if line.x2 > self.max_num : 
            self.max_num = line.x2

In [80]:
def init_img(id) : 
    
    f = open("./result/res_img{}.txt".format(id), 'r')
    image = np.array(Image.open("./img/img{}.jpg".format(id)))

    boxes = []
    box_x = []
    box_y = []
    box_width = []
    box_height = []

    while True:
        line = f.readline()
        if not line: break
        coord = line.strip().split(",")
        coord = [int(item) for item in coord]

        box_width.append(coord[2] - coord[0])
        box_height.append(coord[5] - coord[1])

        box_angle_x = max(abs(coord[4] - coord[2]), abs(coord[6] - coord[0]))
        box_angle_y = max(abs(coord[7] - coord[5]), abs(coord[3] - coord[1]))

        x1 = min(coord[0], coord[6])
        y1 = max(coord[1], coord[3])
        x2 = min(coord[2], coord[4])
        y2 = min(coord[5], coord[7])

        if box_angle_x < 5 or box_angle_y < 5 :
            cv2.rectangle(image,(coord[0], coord[1]),(coord[2],coord[5]),(255,0,0), 2) 
            boxes.append(box(x1, y1, x2, y2))
    
    cv2.imwrite("./img/bounding/img{}/{}_init.jpg".format(id, id), image)
    f.close()
    return boxes 

In [81]:
# 2. 줄 단위로 box 세로 길이 맞추기 

def fixY_img(id, boxes) : 

    image = np.array(Image.open("./img/img{}.jpg".format(id)))
    
    onelines = []
    oneline = box(-1, -1, -1, -1)
    idx = 0

    for i in range(0, len(boxes)) :     
        if i != 0 and (abs(boxes[i].y1 - boxes[i-1].y1) >= (boxes[i].y2 - boxes[i].y1) / 4) :
            onelines.append(oneline)
            for j in range(idx, i) :
                boxes[j].y1 = oneline.y1
                boxes[j].y2 = oneline.y2
            idx = i
            oneline = box(-1, -1, -1, -1)
        elif i != 0 and (boxes[i].y2 - oneline.y1) > 50 and boxes[i].y2 - boxes[i].y1 < 35:
            onelines.append(oneline)
            for j in range(idx, i) :
                boxes[j].y1 = oneline.y1
                boxes[j].y2 = oneline.y2
            idx = i
            oneline = box(-1, -1, -1, -1)

        if boxes[i].x1 < oneline.x1 or oneline.x1 == -1 :
            oneline.x1 = boxes[i].x1
        if boxes[i].y1 < oneline.y1 or oneline.y1 == -1 :
            oneline.y1 = boxes[i].y1
        if boxes[i].x2 > oneline.x2 or oneline.x2 == -1 :
            oneline.x2 = boxes[i].x2
        if boxes[i].y2 > oneline.y2 or oneline.y2 == -1 :
            oneline.y2 = boxes[i].y2

    onelines.append(oneline)
    for j in range(idx, len(boxes)) :
        boxes[j].y1 = oneline.y1
        boxes[j].y2 = oneline.y2

    for boxb in boxes : 
        cv2.rectangle(image,(boxb.x1,boxb.y1),(boxb.x2,boxb.y2),(255,0,0), 2)
    for ol in onelines : 
        cv2.rectangle(image,(ol.x1,ol.y1),(ol.x2,ol.y2),(0,0,255),2)
    
    cv2.imwrite("./img/bounding/img{}/{}_fixY.jpg".format(id, id), image)

    return boxes


In [113]:
#box oneline으로 정렬

def WordToSentence(id, boxes) : 
    boxes = sorted(boxes, key = attrgetter('y2', 'x1'))
    image = np.array(Image.open("./img/img{}.jpg".format(id)))
    sentences = []
    sentence = box(-1, -1, -1, -1)
    box_diff = -1
    idx = 0
    num = 0

    for i in range(0, len(boxes)) :        
        yflag = 0
        if i != 0 and abs(boxes[i].y1 - boxes[i-1].y1) >= boxes[i].y2 - boxes[i].y1 :
            cv2.rectangle(image,(sentence.x1,sentence.y1),(sentence.x2,sentence.y2),(255,0,0), 2)
            sentences.append(sentence)
            sentence = box(boxes[i].x1, boxes[i].y1, boxes[i].x2, boxes[i].y2)
            yflag = 1

        if i != 0 and abs(boxes[i].x1 - boxes[i-1].x2) >= 15 and boxes[i].y2 - boxes[i].y1 < 55: #박스간격이 특정값보다 크면 out
            if yflag == 0 : 
                cv2.rectangle(image,(sentence.x1,sentence.y1),(sentence.x2,sentence.y2),(255,0,0), 2)
                sentences.append(sentence)
                sentence = box(boxes[i].x1, boxes[i].y1, boxes[i].x2, boxes[i].y2)

        if boxes[i].x1 < sentence.x1 or sentence.x1 == -1 :
            sentence.x1 = boxes[i].x1
        if boxes[i].y1 < sentence.y1 or sentence.y1 == -1 :
            sentence.y1 = boxes[i].y1
        if boxes[i].x2 > sentence.x2 or sentence.x2 == -1 :
            sentence.x2 = boxes[i].x2
        if boxes[i].y2 > sentence.y2 or sentence.y2 == -1 :
            sentence.y2 = boxes[i].y2
    
        sentence.children.append(boxes[i])
        
    cv2.rectangle(image,(sentence.x1,sentence.y1),(sentence.x2,sentence.y2),(255,0,0), 2)
    sentences.append(sentence)
    
    
    for st in sentences : 
        for boxx in st.children : 
            print(len(st.children))
            print((boxx.x1,boxx.y1),(boxx.x2,boxx.y2))
            cv2.rectangle(image,(boxx.x1,boxx.y1),(boxx.x2,boxx.y2),(0,0,255), 1)
    
    cv2.imwrite("./img/bounding/img{}/{}_WordToSentence.jpg".format(id, id), image)
    return sentences 


In [83]:
# column 화

def seperate_columns(id, sentences) : 

    headlines = []

    sentences = sorted(sentences, key = attrgetter('y2', 'x1'))
    for i in range(0, len(sentences)) :
        if sentences[i].y2 - sentences[i].y1 > 45 : 
            headlines.append(sentences[i])
            i += 1
        else : break
        
    sentences = sentences[i:]

    columns = []
    idx = 0

    for line in sorted(sentences, key = attrgetter('x1', 'y1')) : 
        if len(columns) == 0 :
            columns.append(column(line.x1, line.x2, idx))
            idx += 1
        elif ((line.x1 + line.x2)/2) > columns[-1].max_num and columns[-1].max_num - line.x1 < 0 : 
            columns.append(column(line.x1, line.x2, idx))
            idx += 1
        columns[-1].add_line(line)

    image = np.array(Image.open("./img/img{}.jpg".format(id)))

    for i in range(0, len(columns)) : 
        for line in columns[i].lines :
            color = (255,255, 255)
            if i % 3 == 0 : color = (225, 0, 0)
            elif i % 3 == 1 : color = (0, 225, 0)
            elif i % 3 == 2 : color = (0, 0, 255)
            cv2.rectangle(image,(line.x1,line.y1),(line.x2, line.y2), color, 2)
    
    for headline in headlines :
        cv2.rectangle(image,(headline.x1,headline.y1),(headline.x2, headline.y2), (0,0,0), 3)
    
    cv2.imwrite("./img/bounding/img{}/{}_seperate_columns.jpg".format(id, id), image)
    
    return headlines, columns

In [122]:
def split_sentence(id, headlines, columns) : 
    image = np.array(Image.open("./img/img{}.jpg".format(id)))
    
    num_sentences = []
    

    for j in range(0, len(headlines)) : 
        idx = 1
        for word in headlines[j].children : 
            oneline_image = image.copy()
            oneline_image = oneline_image[word.y1:word.y2, word.x1:word.x2]
            cv2.imwrite("./img/bounding/sentences/img{}/headline/h{}_{}.png".format(id,j,idx), oneline_image)
            idx+=1
    
    num_sentences.append(len(headlines))
    
    for j in range(0, len(columns)) :
        lines = sorted(columns[j].lines, key = attrgetter('y1', 'y2')) 
        num_sentences.append(len(lines))        
        for i in range(0,len(lines)): 
            idx = 1
            for word in lines[i].children : 
                oneline_image = image.copy()
                oneline_image = oneline_image[word.y1:word.y2, word.x1:word.x2]
    #             cv2.rectangle(image,(lines[i].x1,lines[i].y1),(lines[i].x2, lines[i].y2), (0,0,255))
                cv2.imwrite("./img/bounding/sentences/img{}/column/c{}_s{}_{}.png".format(id, j, i, idx), oneline_image)
                idx+=1
            
    return num_sentences

In [85]:
tmp_idx = 1

In [86]:
def tmp_split_sentence(id, headlines, columns) : 
    image = np.array(Image.open("./img/img{}.jpg".format(id)))
    global tmp_idx

    for headline in headlines : 
        oneline_image = image.copy()
        oneline_image = oneline_image[headline.y1:headline.y2, headline.x1:headline.x2]
        cv2.imwrite("/Users/sangyeob/ku2022-2/deeplearning/final_project/imagedata/printed/data2/raw/train/images/image_{:05d}.png".format(tmp_idx), oneline_image)
        tmp_idx+=1
    
    for j in range(0, len(columns)) :
        lines = sorted(columns[j].lines, key = attrgetter('y1', 'y2')) 
        for i in range(0,len(lines)): 
            oneline_image = image.copy()
            oneline_image = oneline_image[lines[i].y1:lines[i].y2, lines[i].x1:lines[i].x2]
#             cv2.rectangle(image,(lines[i].x1,lines[i].y1),(lines[i].x2, lines[i].y2), (0,0,255))
            cv2.imwrite("/Users/sangyeob/ku2022-2/deeplearning/final_project/imagedata/printed/data2/raw/train/images/image_{:05d}.png".format(tmp_idx), oneline_image)
            tmp_idx+=1
        

In [106]:
def CRAFT_bounding(id) : 
    boxes = init_img(id)
    fixed_boxes = fixY_img(id, boxes)
    sentences = WordToSentence(id, fixed_boxes)
    headlines, columns = seperate_columns(id, sentences)
    num_sentences = split_sentence(id, headlines, columns)
    
    print(num_sentences)

In [88]:
# python test.py --trained_model='./craft_mlt_25k.pth' --test_folder='./images'

In [123]:
for i in range(1, 9) : 
    CRAFT_bounding(i)

5
(31, 29) (153, 103)
5
(165, 29) (320, 103)
5
(330, 29) (569, 103)
5
(579, 29) (699, 103)
5
(712, 29) (838, 103)
4
(18, 176) (135, 214)
4
(138, 176) (199, 214)
4
(202, 176) (292, 214)
4
(295, 176) (358, 214)
3
(18, 254) (174, 285)
3
(176, 254) (290, 285)
3
(292, 254) (417, 285)
4
(19, 293) (179, 326)
4
(179, 293) (234, 326)
4
(237, 293) (336, 326)
4
(338, 293) (417, 326)
1
(19, 333) (120, 364)
5
(42, 372) (143, 404)
5
(146, 372) (201, 404)
5
(202, 372) (255, 404)
5
(259, 372) (311, 404)
5
(315, 372) (417, 404)
4
(18, 412) (69, 443)
4
(72, 412) (244, 443)
4
(247, 412) (361, 443)
4
(364, 412) (415, 443)
2
(19, 450) (268, 483)
2
(268, 450) (417, 483)
3
(19, 490) (194, 523)
3
(191, 490) (245, 523)
3
(244, 490) (348, 523)
3
(450, 490) (504, 523)
3
(504, 490) (678, 523)
3
(679, 490) (849, 523)
4
(42, 529) (179, 562)
4
(181, 529) (259, 562)
4
(262, 529) (363, 562)
4
(364, 529) (415, 562)
3
(450, 529) (518, 562)
3
(521, 529) (716, 562)
3
(721, 529) (849, 562)
5
(18, 569) (69, 600)
5
(72, 569)

[1, 18, 22]
4
(214, 21) (408, 102)
4
(432, 21) (611, 102)
4
(629, 21) (830, 102)
4
(838, 21) (1083, 102)
3
(211, 124) (559, 207)
3
(572, 124) (856, 207)
3
(871, 124) (1002, 207)
3
(13, 266) (115, 299)
3
(120, 266) (323, 299)
3
(327, 266) (408, 299)
3
(441, 266) (543, 299)
3
(546, 266) (650, 299)
3
(653, 266) (836, 299)
4
(869, 266) (899, 299)
4
(899, 266) (1072, 299)
4
(1076, 266) (1155, 299)
4
(1157, 266) (1264, 299)
5
(13, 305) (41, 338)
5
(43, 305) (122, 338)
5
(124, 305) (192, 338)
5
(194, 305) (249, 338)
5
(251, 305) (408, 338)
4
(440, 305) (589, 338)
4
(594, 305) (674, 338)
4
(677, 305) (729, 338)
4
(729, 305) (834, 338)
1
(869, 305) (923, 338)
3
(13, 345) (141, 377)
3
(141, 345) (220, 377)
3
(222, 345) (406, 377)
6
(441, 345) (495, 377)
6
(502, 345) (557, 377)
6
(557, 345) (613, 377)
6
(613, 345) (699, 377)
6
(701, 345) (729, 377)
6
(733, 345) (834, 377)
3
(893, 345) (998, 377)
3
(1002, 345) (1207, 377)
3
(1212, 345) (1264, 377)
5
(13, 384) (65, 417)
5
(67, 384) (168, 417)
5
(17

3
(216, 8) (427, 79)
3
(438, 8) (612, 79)
3
(623, 8) (795, 79)
4
(107, 78) (169, 112)
4
(220, 108) (400, 182)
4
(409, 108) (646, 182)
4
(661, 108) (798, 182)
3
(16, 266) (110, 307)
3
(114, 266) (180, 307)
3
(184, 266) (315, 307)
7
(452, 266) (478, 307)
7
(478, 266) (532, 307)
7
(534, 266) (591, 307)
7
(593, 266) (620, 307)
7
(624, 266) (700, 307)
7
(700, 266) (800, 307)
7
(804, 266) (855, 307)
6
(452, 309) (503, 339)
6
(505, 309) (534, 339)
6
(536, 309) (616, 339)
6
(618, 309) (720, 339)
6
(724, 309) (800, 339)
6
(804, 309) (855, 339)
4
(18, 319) (108, 358)
4
(108, 319) (198, 358)
4
(202, 319) (294, 358)
4
(294, 319) (358, 358)
5
(450, 347) (528, 378)
5
(532, 347) (636, 378)
5
(640, 347) (691, 378)
5
(695, 347) (773, 378)
5
(777, 347) (857, 378)
4
(18, 372) (137, 409)
4
(139, 372) (229, 409)
4
(231, 372) (298, 409)
4
(298, 372) (360, 409)
2
(452, 386) (730, 419)
2
(730, 386) (833, 419)
3
(474, 423) (677, 458)
3
(677, 423) (753, 458)
3
(751, 423) (855, 458)
5
(12, 464) (128, 497)
5
(131